In [1]:

!pip3 install tensorflow opencv-python mediapipe scikit-learn matplotlib


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import cv2 
import numpy as np 
import os # easier to acess files
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic # holistic model - make detections
mp_drawing = mp.solutions.drawing_utils # drawing utilities - draw detections




In [4]:
# image is a photo
# model is an mp_holisitic model

# Steps to use media pipe:
# 1. grab image and convert from bgr to rgb, then unwritable (saves memory)
# 2. make our detection
# 3. convert back to writeable and bgr



def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion
    # cv2Color converts images from one colorspace to another
    image.flags.writeable = False
    results = model.process(image) # make prediction from image grame
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion
    return image, results

In [5]:
# draws landmarks on photos

#draw_landmarks is helper function that makes it easy to draw onto image
# needs image, landmarks, and what type of connections you want to use
# POSE and FACE connections will provide map for different landmarks
# NOTE: applies landmark drawings to image in place


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw right hand connections


In [6]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) # draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) # draw face connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) # draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [29]:
# # code to access openCV


# cap = cv2.VideoCapture(0) # access video cam on device port 0

# # with is used to handle resource management
# # set mediapipe model

# # min detection is initial detection, tracking confidence is preceding tracking confidence
# # if you want higher inital confidence in answer, then increase it and vice versa
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
# 	while cap.isOpened(): # while camera is on

# 		# Read feed (reading frame from webcam)
# 		ret, frame = cap.read()

# 		# make detections
# 		image, results = mediapipe_detection(frame, holistic)
# 		print(results)

# 		# draw landmarks
# 		draw_styled_landmarks(image, results)

# 		# show frame to screen
# 		cv2.imshow('OpienCV Feed', image)
		
# 		# break gracefully
# 		if cv2.waitKey(1) & 0xFF == ord('q'):
# 			break

# # release cv2 and close all windows
# cv2.destroyAllWindows()
# cv2.waitKey(1)
# cap.release()

In [30]:
# draw_landmarks(frame, results)

In [31]:

# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [32]:
# # diffe
# results.face_landmarks

Extract keypoint values

In [33]:
# pose = []
# for res in results.pose_landmarks.landmark:
#     test = np.array([res.x, res.y, res.z, res.visibility])
#     pose.append(test)
# for loop way of below code

In [34]:
# pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeroes(132)
# # makes pose array with x,y,z and visibility, and makes list of lists into just a big list

# # each hand has 21 landmarks; each landmark has x, y and z coordinate values each; thus empty arrays (when hand not visible) should be 21*3=63
# lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
# rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

# face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # makes pose array with x,y,z and visibility, and makes list of lists into just a big list

    # each hand has 21 landmarks; each landmark has x, y and z coordinate values each; thus empty arrays (when hand not visible) should be 21*3=63
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose, face, lh, rh])

In [36]:
# extract_keypoints(results)

In [8]:
# path for exported data, numpy arrays
DATA_PATH = os.path.join("MP_Data")

#Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# thirty videos with of data
no_sequences = 30

# videos are going to be 30 frames of length
sequence_length = 30

#1662 datapoints, 30 

In [9]:
# for each action we have a folder
# hello
# - each action has no_sequence folders 
# - 0 (frame 1, frame 2, ... Frame 30)
# - 1
# - 2
# ... 
# - 29
# - - each numbered folder will have 30 frames (30 arrays) (sequence lenght)


# thanks
# i love you


for action in actions:
    # for each action
    for sequence in range(no_sequences):
        try:
            # if folder already exist, will pass, else make numbered folders in action
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [39]:
# # code to access openCV


# cap = cv2.VideoCapture(0) # access video cam on device port 0

# # with is used to handle resource management
# # set mediapipe model

# # min detection is initial detection, tracking confidence is preceding tracking confidence
# # if you want higher inital confidence in answer, then increase it and vice versa
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
	
#     # replacing while cap is open
#     for action in actions:
#           for sequence in range(no_sequences):
#                 for frame_num in range(sequence_length):
#                             # Read feed (reading frame from webcam)
#                             ret, frame = cap.read()

#                             # make detections
#                             image, results = mediapipe_detection(frame, holistic)
#                             print(results)

#                             # draw landmarks
#                             draw_styled_landmarks(image, results)

                            
                            
#                             # apply weight logic
#                             if frame_num == 0:
#                                   # image, text, position, font, font size, color
#                                   cv2.putText(image, 'Start collection', (120, 200), 
#                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255,0), 5, cv2.LINE_AA)
#                                   cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 50), 
#                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255,0), 5, cv2.LINE_AA)
#                                   cv2.waitKey(2000) # 2 second break
#                             else:
#                                   cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 50), 
#                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255,0), 5, cv2.LINE_AA)

#                             # show frame to screen
#                             cv2.imshow('OpenCV Feed', image)

#                             # new -- export keypoints
#                             keypoints = extract_keypoints(results)
#                             npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                             np.save(npy_path, keypoints)
                            
                            
#                             # gracefully break
#                             if cv2.waitKey(1) & 0xFF == ord('q'):
#                                   break

# # release cv2 and close all windows
# cv2.destroyAllWindows()
# cv2.waitKey(1)
# cap.release()

In [85]:
cap.release()
cv2.destroyAllWindows()

: 

In [10]:
from sklearn.model_selection import train_test_split # used for training and testing
from tensorflow.keras.utils import to_categorical # used to make labels

In [11]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [12]:
sequences, labels = [], []
# sequences are feature data (X), labels are target data (y)
for action in actions:
    for sequence in range(no_sequences):
        window = [] # all frames for specific sequence (video)
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # loads numpy frame 0, frame 1, .. 
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        # append 

In [13]:
X = np.array(sequences) # makes to np array
y = to_categorical(labels).astype(int) # uses one hot encoding to prevent bias


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) # splits data

In [15]:
from tensorflow.keras.models import Sequential # Sequential lets you build a sequential NN
from tensorflow.keras.layers import LSTM, Dense # LSTM is temporal (involves time) and lets build model
from tensorflow.keras.callbacks import TensorBoard # allows to logging in tensor board

In [16]:
# Tensorboard is webapp to see neural network training

log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
model = Sequential() # easy to make neural network

# 3 sets of lstm levels 
# first adds 64 neurons, and if it needs to pass to next level return seqs must be true, 30 frames with 1662 datapoints
# lstm layer is good for long term sequences
# dont return lstm layer on last layer
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

# actions will turn output into 3 layers, and softmax will show probabiliies from sum

In [18]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["categorical_accuracy"])
# loss function must be categorical (when multiclass data), binary class uses binary entropy, regression is mean square error

In [23]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
3/3 [==============================] - 1s 69ms/step - loss: 1.1916 - categorical_accuracy: 0.6824
Epoch 2/500
3/3 [==============================] - 0s 70ms/step - loss: 1.2030 - categorical_accuracy: 0.6588
Epoch 3/500
3/3 [==============================] - 0s 57ms/step - loss: 1.5580 - categorical_accuracy: 0.7294
Epoch 4/500
3/3 [==============================] - 0s 56ms/step - loss: 1.1190 - categorical_accuracy: 0.6706
Epoch 5/500
3/3 [==============================] - 0s 62ms/step - loss: 1.1914 - categorical_accuracy: 0.7412
Epoch 6/500
3/3 [==============================] - 0s 57ms/step - loss: 0.8251 - categorical_accuracy: 0.7176
Epoch 7/500
3/3 [==============================] - 0s 62ms/step - loss: 0.8794 - categorical_accuracy: 0.7294
Epoch 8/500
3/3 [==============================] - 0s 62ms/step - loss: 0.9001 - categorical_accuracy: 0.7529
Epoch 9/500
3/3 [==============================] - 0s 62ms/step - loss: 0.5387 - categorical_accuracy: 0.7647
Epoch 10/5

KeyboardInterrupt: 

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 596675 

In [25]:
res = model.predict(X_test)
actions[np.argmax(res[3])]

1/1 [==============================] - 0s 30ms/step


'thanks'

In [26]:
actions[np.argmax(y_test[3])]

'thanks'

In [27]:
model.save('action.h5') # saves weights

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [30]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 39ms/step


In [31]:
ytrue = np.argmax(y_test, axis=1).tolist() # conv
yhat = np.argmax(yhat, axis =1).tolist()

In [32]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 1],
        [1, 1]],

       [[3, 0],
        [1, 1]],

       [[3, 1],
        [0, 1]]])

In [37]:
accuracy_score(ytrue, yhat)

0.6

: 

In [36]:
# needs 30 frames to make a movement
sequence = []
sentence = []
threshold = 0.4

# code to access openCV

cap = cv2.VideoCapture("../uploads/video2.mp4")  # Access video file # access video cam on device port 0

# with is used to handle resource management
# set mediapipe model

# min detection is initial detection, tracking confidence is preceding tracking confidence
# if you want higher inital confidence in answer, then increase it and vice versa
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
	while cap.isOpened(): # while camera is on

		# Read feed (reading frame from webcam)
		ret, frame = cap.read()

		# make detections
		image, results = mediapipe_detection(frame, holistic)
		# print(results)

		# draw landmarks
		draw_styled_landmarks(image, results)
		
        # 2. prediction logic
		keypoints = extract_keypoints(results)
		sequence.insert(0,keypoints)
		sequence = sequence[:30]
		
		if (len(sequence) == 30):
			res = model.predict(np.expand_dims(sequence, axis=0))[0]
			# expand dims allows us to test one sequence since its expecting (0, 30, 1662)
			print(actions[np.argmax(res)])

		# show frame to screen
		cv2.imshow('OpenCV Feed', image)
		
		# break gracefully
		if cv2.waitKey(1) & 0xFF == ord('q'):
			break

# release cv2 and close all windows
cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()

I0000 00:00:1705807253.556665       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


1/1 [==============================] - 0s 22ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 20ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 20ms/step
thanks
1/1 [==============================] - 0s 20ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 20ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 20ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thank

error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
